<div style="background: linear-gradient(to right, #56BEB7 0%, #56BEB7 50%, #2B317F 50%, #2B317F 100%);"><br></div>

<img src="oathlink_logo.png" width="150" align="right"/>

# Tutorial Oathlink

## Flujo de información

La ley 19.628 sobre la protección de la vida privada define lineamientos para la implementación del Sistema de Finanzas Abiertas (SFA) intoducido por la ley Fintech.

Crucialmente el cliente final (persona natural o juridica) es el dueño de sus datos.

Las instituciones financieras (tales como los bancos) son meros custodios de dicha información.

En consequencia, el dueño tiene el derecho de solicitarle al custodio la entrega de su informacion personal y potencialmente sensible, por ejemplo los movimientos de su cuenta corriente a un usuario (tipicamente a una Fintech) que le brindaría un servicio basado en est información.

Oathlink interviene en la entrega de estos datos personales de parte del custodio al usuario, garantizando minimos de seguidad y velando el cumplimiento con la ley 19.628.

<center><img src="information_flow_chart.png" width="500" align="middle"/></center>

## I. Hello world!

### I.A. Archivos entregados por Oathlink

El equipo de Oathlink confirma la inscripción de una nueva institución (sea custodio o usuario) con la entrega de los tres archivos siguientes:

- *certificate.pem y certificate.key*: Certificados público y privado de autenticación de protocolo Mutual TLS.
- *secret.key*: Clave de encriptación privada para el algoritmo Fernet.

### I.B. Instalación del cliente Oathlink

El protocolo Oathlink se utiliza cómodamente mediante una librería en Python que se instala mediante el comando:

In [ ]:
!pip install oathlink

Es importante notar que el carácter "!" antes del comando es exclusivo para ejecutar comandos desde el ambiente Jupyter, en una consola estándar no es necesario incluirlo.

### I.C. Comprobación de la conexión

Para empezar, se importa el paquete Oathlink:

In [ ]:
import oathlink

Luego, se especifican las rutas de los certificados Mutual TLS:

In [ ]:
certificate_filename_pem = '/path/certificate.pem'
certificate_filename_key = '/path/certificate.key'

Finalmente, se comprueba la conexión:

In [ ]:
hello = oathlink.hello(certificate_filename_pem=certificate_filename_pem, certificate_filename_key=certificate_filename_key)
print(hello)

Un mensaje de saludo de parte de Oathlink confirma la conexión. De lo contrario, el mensaje indica la naturaleza del error.

## II. Entrega de la información por el custodio

El custodio es típicamente una institución financiera o bancaria que debe entregar cierta información sensible a un usuario con la autorización previa del dueño de la información.

El protocolo de Oathlink garantiza que la información sea cargada únicamente por el custodio y descargada solamente por el usuario designado.

### II.A. Solicitud del link de carga

El primer paso es la solicitud de un link seguro para cargar la información.

El dueño de la información (owner) y el usuario (user) que recibirá la información se identifican mediante sus RUT (formato 12345678-9):

In [ ]:
owner_id = "RUT_dueño"
user_id = "RUT_usuario"

En cumplimiento de la ley sobre la Protección de la vida privada, el custodio indica la autorización del dueño de la información, el motivo y el propósito del requerimiento(intent) y el tipo de datos que se transmiten (description):

In [ ]:
owner_authorization = "autorización_del_dueño"
description = "tipo_de_datos"
intent = "motivo_y_proposito"

Oathlink registra la iniciación del intercambio de información entre el custodio y el usuario con el comando:

In [ ]:
response_upload = oathlink.upload(certificate_filename_pem=certificate_filename_pem, 
                                  certificate_filename_key=certificate_filename_key, user_id=user_id, owner_id=owner_id, 
                                  owner_authorization=owner_authorization, description=description, intent=intent)
print(response_upload)

import json

response_upload = json.loads(response_upload)
oath_link = response_upload.get("oath_link")
record_id = response_upload.get("record_id")
record_id_encrypted = response_upload.get("record_id_encrypted")

El custodio entonces recibe:
- oath_link: Link de carga seguro que garantiza la trazabilidad del intercambio de información (e.g. "https://secure.oathlink.com/VnKd7mtedcsrGPGWTdYD-Q").
- record_id: Identificador del intercambio que podrá ser utilizado para realizar el seguimiento del intercambio (e.g. "fca2bf15-9f4d-4ede-98f6-c9982493cdb3").
- record_id_encrypted: Identificador encriptado con la clave Fernet del usuario (e.g. "gAAAAABkkLVc5qrtq6O6Su7kyhCq9u-2kwh9NE-SuK9QpzG9yHk-WjzHJnNdgAH2NsGS6cIcGqdwUme7VPl6B9Qp9wLxDWmbP-YJ3-LoPJ-aZTlokUQ7SLrFFip0LvU6dooTc_f0az2H").

El custodio deberá enviar el identificador record_id_encrypted al usuario, quien realizará el procedimiento de descarga detallado en el capítulo siguiente.

Este identificador puede enviarse antes de realizar la carga de la información para aprovechar la asincronicidad del protocolo Oathlink.

### II.B. Carga de la información

El segundo paso es la carga de la información al link seguro oath_link mediante el protocolo PUT (HTTP). La carga de la información está habilitada solamente a partir de los IP previamente declarados.

La información cargada estará disponible solamente 30 segundos para su descarga por el usuario. Después de este plazo, la información se borrará automáticamente y Oathlink respaldará esta expiración.

La variable payload es la ruta del archivo que contiene la información sensible o bien directamente la información como string:

In [ ]:
data = 'path/data.txt'

Si bien es posible realizar la carga directamente con un protocolo PUT, la librería proporciona una función de carga:

In [ ]:
upload = oathlink.data_upload(oath_link=oath_link, data=data)
print(upload)

### II.C. Cancelar el intercambio

En caso de emergencia o equivocación, el custodio puede cancelar el envío de información mientras no haya sido accedida por el usuario.

El custodio indica el identificador record_id:

In [ ]:
cancel = oathlink.cancel(certificate_filename_pem=certificate_filename_pem, certificate_filename_key=certificate_filename_key, 
                         record_id=record_id)
print(cancel)

## III. Recepción de la información por el usuario

El usuario es habitualmente un proveedor de servicios tal como una Fintech.

El protocolo de Oathlink aplica estándares de ciberseguridad y facilita el cumplimiento con la ley sobre Protección de la vida privada, generando confianza entre el usuario y el custodio de la información.

### III.A. Desencriptación del identificador

El custodio no entrega la información, sino el identificador encriptado del intercambio record_id encrypted.

Es posible que el custodio haya enviado el identificador antes de haber concluido la carga de la información en sí, pues el protocolo Oathlink es asincrónico.

El primer paso es la desencriptación gracias a la clave Fernet (e.g. "hBLUHczSeJbDXBIsSgAIs2mYWCsVaYLoSb2yoK95OLE=") que se encuentra en el archivo "secret.key" para obtener el record_id. Esta desencriptación puede alternativamente realizarse con cualquier librería genérica del algoritmo Fernet. El cliente Oathlink contiene tal función de desencriptación:

In [ ]:
secret = 'path/secret.key'
record_id_decrypted = oathlink.decrypt(record_id_encrypted=record_id_encrypted, secret=secret)
print(record_id_decrypted)

Es importante notar que en el argumento "secret" se puede especificar tanto la clave como un string o la ruta hacia el archivo ".key" que contiene la clave.

### III.B. Solicitud del link de descarga

El segundo paso es la solicitud del link de descarga correspondiente al identificador record_id:

In [ ]:
oath_link = oathlink.download(certificate_filename_pem=certificate_filename_pem, 
                              certificate_filename_key=certificate_filename_key, record_id=record_id)

El usuario recibe:
- oath_link: Link de descarga segura por protocolo GET (e.g. "https://secure.oathlink.com/cxoQYHLj0PLhdYoYaOO6Zg").

Oathlink registra el acceso del usuario a la información solicitada. A partir de este punto, el usuario se vuelve responsable del cumplimiento con la ley sobre Protección de la vida privada. En caso de no acceder a la información dentro de un plazo de 30 segundos, esta se borrará automáticamente y ya no será accesible.

### III.C. Descarga de la información

El tercer paso es la descarga de la información desde el link seguro oath_link mediante el protocolo GET (HTTP). La descarga de la información está habilitada solamente a partir de los IP previamente declarados.

El usuario puede descargar la información con este link de manera directa. No obstante, el cliente proporciona una función de descarga para facilitar el manejo de la asincronicidad del protocolo Oathlink:

In [ ]:
data = oathlink.data_download(oath_link = oath_link)
print(data)

La variable data contiene el nombre del archivo con la información solicitada.


### III.D. Eliminación de la información

En cumplimiento de la ley, el usuario deberá eliminar la información de sus sistemas cuando su uso haya caducado.

El cuarto paso es la declaración de esta eliminación:

In [ ]:
deletion = oathlink.delete(certificate_filename_pem=certificate_filename_pem, certificate_filename_key=certificate_filename_key, 
                           record_id=record_id)
print(deletion)


Oathlink registra esta declaración y concluye el intercambio. El historial del intercambio se archiva y permanece disponible para eventuales auditorías.

## IV. Trazabilidad y auditoría

El protocolo Oathlink les permite a los custodios y usuarios realizar seguimientos y auditar sus intercambios de información y así detectar eventuales irregularidades a las leyes y normas vigentes.
Todo intercambio se encuentra en uno de los estados siguientes:

- Creado: El intercambio fue iniciado por el custodio pero el usuario aún no accedió a la información.

- Accedido: El usuario accedió a la información y se ha vuelto responsable de los cumplimientos legales y normativos.

- Destruido: El usuario declaró haber borrado la información de sus sistemas. Este estado es la conclusión normal de un intercambio.

- Expirado: El usuario no ha accedido a la información dentro de un plazo de 30 segundos, la información fue borrada del sistema de Oathlink y el intercambio expiró.

- Cancelado: El custodio decidió cancelar el intercambio antes que el usuario haya accedido a la información.

### IV.A. Trazabilidad de un intercambio

Tanto el custodio como el usuario puede consultar el historial detallado y el estado de un intercambio mediante el identificador record_id con el comando:

In [ ]:
record = oathlink.record(certificate_filename_pem=certificate_filename_pem, certificate_filename_key=certificate_filename_key, 
                         record_id=record_id)
print(record)

Este informe contiene toda la metadata relacionada al intercambio especificado, en particular los datos de las conexiones (IP, fechas, etc.), con el fin de trazar y auditarlo en detalle.

### IV.B. Intercambios vigentes

Oathlink mantiene el listado de los intercambios vigentes, o sea que se encuentran en el estado creado o accedido, con el comando:

In [ ]:
outstanding = oathlink.outstanding(certificate_filename_pem=certificate_filename_pem, 
                                   certificate_filename_key=certificate_filename_key)
print(outstanding)

Este listado le permite al custodio realizarles un seguimiento a sus usuarios. Así, el custodio puede exigirles a sus usuarios borrar la información entregada.

También, los usuarios pueden verificar a todo instante que están en pleno cumplimiento.

Este listado entrega para cada contraparte:

- record_id: Identificador del intercambio.

- date_time: Fecha y hora UTC de iniciación del intercambio.

- status: Estado actual.

- custodian_id: Identificador (RUT) del custodio.

- user_id: Identificador (RUT) del usuario.

- owner_id: Identificador (RUT) del dueño de la información.